In [1]:
from dataloader import Dataset
from utils import * 

data = Dataset("Stage_1_Data_ver_4/stage1/box")
data.load_data_new()
data.load_data()

In [5]:
images_to_add, appended_ids = data.computeSceneGraph()

In [4]:
data.visibility[:,40].sum()

1104.0

In [18]:
((data.visibility[:,40] == 1) == np.any(data.visibility_new[:,40],axis = 1)).all()

True

In [ ]:
from scipy.spatial.transform import Rotation 
from scipy.sparse import lil_matrix
import time
from scipy.optimize import least_squares
def computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids):

    X_indices = (reconstructed_ind == 1)[:,0]
    current_visibility = visibility[:,appended_ids[:num_views]][X_indices]
    
    return X_indices, current_visibility

def computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility):
    
    current_2d_points = points_2d[:, appended_ids[:current_visibility.shape[1]]][X_indices]
    point_indices, camera_indices = current_visibility.nonzero()
    return current_2d_points[point_indices, camera_indices], point_indices, camera_indices
    
def projectFromCameras(X, cam_params, camera_indices, point_indices, K):

    projected = np.zeros((X.shape[0], cam_params.shape[0], 3))

    for camera in range(cam_params.shape[0]):

        R = Rotation.from_rotvec(cam_params[camera,:3]).as_matrix()
        C = cam_params[camera, 3:]
        P = computeProjectionMatrix(K, R, C)
        
        projected[:, camera] = projectPoint(P, X)

    return projected[point_indices, camera_indices][:,:2]

def objective(x0, num_views, n_points, camera_indices, point_indices, points_2d, K):
    
    camera_params = x0[:num_views * 6].reshape((num_views, 6))
    points_3d = x0[num_views * 6:].reshape((n_points, 3))
    points_proj = projectFromCameras(points_3d, camera_params,camera_indices,point_indices,K)
    error_vec = (points_proj - points_2d).ravel()

    return error_vec

def bundle_adjustment_sparsity(num_views, n_points, camera_indices, point_indices):
    m = camera_indices.size * 2
    n = num_views * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)
    for s in range(6):
        A[2 * i, camera_indices * 6 + s] = 1
        A[2 * i + 1, camera_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, num_views * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, num_views * 6 + point_indices * 3 + s] = 1

    return A

def bundleAdjustment(X_reconstructed, reconstructed_ind, points_2d, visibility, camera_rotations, camera_translations, K, num_views, appended_ids):
    
    X_indices, current_visibility = computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids)
    X_3d = X_reconstructed[X_indices]
    n_points = X_3d.shape[0]
    X_2d,point_indices, camera_indices = computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility)

    cam_param_list = []
    for i in range(num_views):
        R, C = camera_rotations[i], camera_translations[i]
        rotvec = Rotation.from_matrix(R).as_rotvec()
        RC = [rotvec[0], rotvec[1], rotvec[2], C[0], C[1], C[2]]
        cam_param_list.append(RC)

    cam_param_list = np.array(cam_param_list).reshape(-1, 6)

    x0 = np.hstack((cam_param_list.ravel(), X_3d.ravel()))

    A = bundle_adjustment_sparsity(num_views,n_points, camera_indices, point_indices)
    res = least_squares(objective, x0, jac_sparsity=A, verbose=0, x_scale='jac', max_nfev=100, method='trf',
                        args=(num_views, n_points, camera_indices, point_indices,X_2d, K))
    
    optimized_params = res.x
    optimized_RC = optimized_params[:num_views * 6].reshape((num_views, 6))
    optimized_3d = optimized_params[num_views * 6:].reshape((n_points, 3))

    optimized_X = np.zeros_like(X_reconstructed)
    optimized_X[X_indices] = optimized_3d

    optimized_camera_rotations, optimized_camera_translation = [], []
    for i in range(len(optimized_RC)):
        R = Rotation.from_rotvec(optimized_RC[i, :3]).as_matrix()
        C = optimized_RC[i, 3:].reshape(3,1)
        optimized_camera_translation.append(C)
        optimized_camera_rotations.append(R)

    return optimized_X, optimized_camera_rotations, optimized_camera_translation


In [ ]:
images_to_add, appended_ids = data.computeSceneGraph()

In [ ]:
images_to_add

In [ ]:
appended_ids = np.arange(50).tolist()

In [ ]:

indices    = np.logical_and(visibility[:,appended_ids[0]], visibility[:,appended_ids[1]])
pts1, pts2 = data.points[indices, appended_ids[0]], data.points[indices, appended_ids[1]]
E, best_criterion = estimate_essential_matrix(pts1, pts2, data.K)
rotations, translations = extract_cam_pose(E, data.K)
rot, trans = decomposeEssentialMat(E)
points_3d = []
for i in range(4):
    P1 = computeProjectionMatrix(data.K, np.identity(3), np.zeros((3,)))
    P2 = computeProjectionMatrix(data.K, rotations[i], translations[i])
    points_3d.append(linearTriangulation(pts1, pts2, P1, P2))

X,R,C = cheiralityCondition(points_3d, rotations, translations)

X_reconstructed   = np.zeros((data.points.shape[0], 3))
reconstructed_ind = np.zeros((data.points.shape[0], 1))

X_reconstructed[indices] = X
reconstructed_ind[indices] = 1
reconstructed_ind[(X_reconstructed[:,2] < 0)] = 0

camera_rotations    = []
camera_translations = []
camera_rotations.append(np.eye(3))
camera_rotations.append(R)
camera_translations.append(np.zeros((3,)))
camera_translations.append(C)


In [ ]:
for next_image in range(2, len(data.images)):
    print(next_image)
    visibility_points = data.visibility[:, appended_ids[next_image]][...,None]
    new_points        = data.points[:,appended_ids[next_image]]
    indices           = np.logical_and(reconstructed_ind, visibility_points)[:,0]
    print(indices.sum())
    X_new       = X_reconstructed[indices]
    corr_points = new_points[indices]
    R_new, C_new = DLT_PnP(corr_points, X_new, data.K)
    camera_rotations.append(R_new)
    camera_translations.append(C_new[:,0])
    for previous_images in range(next_image):
        
        indices = np.logical_and(np.logical_and(visibility[:,appended_ids[previous_images]][...,None], 
                                                visibility[:,appended_ids[next_image]][...,None]), 1 - reconstructed_ind)[:,0]
        pts1, pts2 = data.points[indices, appended_ids[previous_images]], data.points[indices, appended_ids[next_image]]
        P1 = computeProjectionMatrix(data.K, camera_rotations[previous_images], camera_translations[previous_images])
        P2 = computeProjectionMatrix(data.K, R_new, C_new)
        points_3d = linearTriangulation(pts1, pts2, P1, P2)
        X_reconstructed[indices]   = points_3d
        reconstructed_ind[indices] = 1
    
    reconstructed_ind[(X_reconstructed[:,2] < 0)] = 0
    X_reconstructed,camera_rotations,camera_translations = bundleAdjustment(X_reconstructed,reconstructed_ind,
                                                                            data.points,data.visibility,camera_rotations,
                                                                            camera_translations, data.K, next_image +1, appended_ids)
    

    

In [ ]:
import numpy as np


def ProjectionMatrix(R,C,K):
    C = np.reshape(C, (3, 1))        
    I = np.identity(3)
    P = np.dot(K, np.dot(R, np.hstack((I, -C))))
    return P

def Lin_tri(K, C1, R1, C2, R2, x1, x2):

    I = np.identity(3)
    C1 = np.reshape(C1, (3, 1))
    C2 = np.reshape(C2, (3, 1))

    P1 = np.dot(K, np.dot(R1, np.hstack((I, -C1))))
    P2 = np.dot(K, np.dot(R2, np.hstack((I, -C2))))

    print(P1, P2)
    p1T = P1[0,:].reshape(1,4)
    p2T = P1[1,:].reshape(1,4)
    p3T = P1[2,:].reshape(1,4)

    p_dash_1T = P2[0,:].reshape(1,4)
    p_dash_2T = P2[1,:].reshape(1,4)
    p_dash_3T = P2[2,:].reshape(1,4)

    all_X = []
    for i in range(x1.shape[0]):
        x = x1[i,0]
        y = x1[i,1]
        x_dash = x2[i,0]
        y_dash = x2[i,1]


        A = []
        A.append((y * p3T) -  p2T)
        A.append(p1T -  (x * p3T))
        A.append((y_dash * p_dash_3T) -  p_dash_2T)
        A.append(p_dash_1T -  (x_dash * p_dash_3T))

        A = np.array(A).reshape(4,4)
        _, _, vt = np.linalg.svd(A)
        v = vt.T
        x = v[:,-1]
        
        all_X.append(x)
    return np.array(all_X)

def PnPError(x, X, R, C, K):
    u,v = x
    X = to_homogeneous(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
    C = C.reshape(-1, 1)
    P = ProjectionMatrix(R,C,K)
    p1, p2, p3 = P
        
    u_proj = np.divide(p1.dot(X) , p3.dot(X))
    v_proj =  np.divide(p2.dot(X) , p3.dot(X))

    x_proj = np.hstack((u_proj, v_proj))
    x = np.hstack((u, v))
    e = np.linalg.norm(x - x_proj)
#     e = np.sqrt(np.square(u - u_proj) + np.square(v - v_proj))
    return  e

def PnPRANSAC(K, features, x3D, n_iterations = 1000, error_thresh = 5):

    inliers_thresh = 0
    chosen_indices = []
    chosen_R, chosen_t = None, None
    n_rows = x3D.shape[0]
    
    for i in range(0, n_iterations):
        
        #select 6 points randomly
        random_indices = np.random.choice(n_rows, size=6)
        X_set, x_set = x3D[random_indices], features[random_indices]
        
        R,C = PnP(X_set, x_set, K)
        
        indices = []
        errors = []
        if R is not None:
            for j in range(n_rows):
                feature = features[j]
                X = x3D[j]
                error = PnPError(feature, X, R, C, K)

                if error < error_thresh:
                    indices.append(j)
                    
        if len(indices) > inliers_thresh:
            inliers_thresh = len(indices)
            chosen_indices = indices
            chosen_R = R
            chosen_t = C
            
    #     filtered_features = features[chosen_indices, :]
    return chosen_R, chosen_t


def reprojectionErrorPnP(x3D, pts, K, R, C):
    P = ProjectionMatrix(R,C,K)
    
    Error = []
    for X, pt in zip(x3D, pts):

        p_1T, p_2T, p_3T = P# rows of P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)
        X = to_homogeneous(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
        ## reprojection error for reference camera points 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        E = np.square(v - v_proj) + np.square(u - u_proj)

        Error.append(E)

    mean_error = np.mean(np.array(Error).squeeze())
    return mean_error

def PnP(X_set, x_set, K):
    N = X_set.shape[0]
    
    X_4 = to_homogeneous(X_set)
    x_3 = to_homogeneous(x_set)
    
    # normalize x
    K_inv = np.linalg.inv(K)
    x_n = K_inv.dot(x_3.T).T
    
    for i in range(N):
        X = X_4[i].reshape((1, 4))
        zeros = np.zeros((1, 4))
        
        u, v, _ = x_n[i]
        
        u_cross = np.array([[0, -1, v],
                            [1,  0 , -u],
                            [-v, u, 0]])
        X_tilde = np.vstack((np.hstack((   X, zeros, zeros)), 
                            np.hstack((zeros,     X, zeros)), 
                            np.hstack((zeros, zeros,     X))))
        a = u_cross.dot(X_tilde)
        
        if i > 0:
            A = np.vstack((A, a))
        else:
            A = a
            
    _, _, VT = np.linalg.svd(A)
    P = VT[-1].reshape((3, 4))
    R = P[:, :3]
    U_r, D, V_rT = np.linalg.svd(R) # to enforce Orthonormality
    R = U_r.dot(V_rT)
    
    C = P[:, 3]
    C = - np.linalg.inv(R).dot(C)
    
    if np.linalg.det(R) < 0:
        R = -R
        C = -C
        
    return R, C

In [ ]:
np.logical_and(data.visibility[:,37], data.visibility[:,40]).sum()

In [ ]:
import  struct
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
                                        rgb_points[i,2].tostring())))
    fid.close()


In [ ]:
X = X_reconstructed[(reconstructed_ind == 1)[...,0]]
c = data.colors[(reconstructed_ind == 1)[...,0]]

In [ ]:
write_pointcloud("initial_attempt.ply", X, c)

In [ ]:
R,t = PnPRANSAC(data.K, corr_points, X_new)

In [3]:
indices = (visibility[:,37, 40] == 1)
print(indices.sum())
pts1, pts2 = data.points[indices, 37], data.points[indices, 40]
    

481


In [4]:
pts1

array([[ 649.27954102,  866.50494385],
       [ 640.56237793,  912.52606201],
       [ 603.8092041 , 1023.90893555],
       [ 645.1864624 ,  880.84533691],
       [ 597.38928223,  684.10150146],
       [ 726.18695068,  684.89654541],
       [ 529.60015869, 1026.57336426],
       [ 346.0562439 , 1033.45483398],
       [ 521.09594727, 1018.80834961],
       [ 504.40634155, 1191.77075195],
       [ 584.24084473,  681.12744141],
       [ 314.0713501 , 1078.24987793],
       [ 447.72103882, 1101.46765137],
       [ 429.86981201,  635.96942139],
       [ 447.86221313,  654.4609375 ],
       [ 423.95281982,  644.81793213],
       [ 437.86862183,  638.21398926],
       [ 441.08880615,  665.1484375 ],
       [ 436.78268433,  988.53540039],
       [ 626.58398438,  915.80847168],
       [ 456.61404419, 1041.01330566],
       [ 444.06286621,  979.31634521],
       [ 503.91680908,  698.41088867],
       [ 558.0970459 ,  811.07269287],
       [ 441.84313965, 1031.47583008],
       [ 558.8059082 ,  7

In [13]:
pts2

array([[ 632.72515869,  839.74102783],
       [ 618.14813232,  886.3828125 ],
       [ 570.75335693,  993.703125  ],
       [ 626.77307129,  854.1930542 ],
       [ 608.24420166,  624.01428223],
       [ 732.20477295,  652.70013428],
       [ 489.63946533,  994.11737061],
       [ 289.19555664,  994.82629395],
       [ 480.69839478,  986.51184082],
       [ 465.22616577, 1135.61169434],
       [ 595.79998779,  619.07562256],
       [ 253.73789978, 1036.61901855],
       [ 397.02679443, 1062.22790527],
       [ 455.75268555,  534.3717041 ],
       [ 471.45474243,  558.7434082 ],
       [ 449.35015869,  543.07745361],
       [ 463.13201904,  538.5869751 ],
       [ 463.48217773,  569.64526367],
       [ 389.2315979 ,  955.01202393],
       [ 604.40039062,  886.45050049],
       [ 408.9347229 , 1006.44610596],
       [ 396.79244995,  945.85321045],
       [ 518.1696167 ,  621.34191895],
       [ 555.54742432,  759.09173584],
       [ 392.23352051,  996.06585693],
       [ 558.36413574,  7

In [ ]:
np.logical_and(visibility[:,appended_ids[0]][...,None], visibility[:,appended_ids[2]][...,None]).sum()

In [ ]:
np.unique(data.points[:,37], axis = 0)

In [ ]:
computeProjectionMatrix(data.K,R,C)

In [ ]:
def test(pts1, pts2, projection_matrix1, projection_matrix2):

    x_prime = to_homogeneous(pts1)
    x       = to_homogeneous(pts2)
    cross_prime = skew_symmetric_matrix(x_prime)
    cross_x     = skew_symmetric_matrix(x)
    
    constraint_x_prime = np.einsum("bij,jk->bik", cross_prime, projection_matrix1)
    constraint_x       = np.einsum("bij,jk->bik", cross_x, projection_matrix2)
    constraints = np.concatenate([constraint_x_prime[:,:2,:], constraint_x[:,:2,:]], axis = 1)
    U,S,V = np.linalg.svd(constraints)
    points_3d = V[:,-1] 
    
    return points_3d,constraints

In [ ]:
q, A = test(pts1, pts2, P1, computeProjectionMatrix(data.K,R,C))

In [ ]:
(new / new[:,3][...,None]).min()

In [ ]:
new = Lin_tri(data.K, camera_translations[0], camera_rotations[0], C,R,pts1, pts2)


In [ ]:
q == new

In [ ]:
def PM(K,R,C):

    C = np.reshape(C, (3, 1))        
    I = np.identity(3)
    P = np.dot(K, np.dot(R, np.hstack((I, -C))))

    return P


In [ ]:
computeProjectionMatrix(data.K, R,C)

In [ ]:
PM(data.K, R,C)

In [ ]:
ProjectionMatrix(R,C,data.K)

In [ ]:
import scipy.optimize as optimize
def NonLinearTriangulation(K, pts1, pts2, x3D, R1, C1, R2, C2):
    """    
    K : Camera Matrix
    pts1, pts2 : Point Correspondences
    x3D :  initial 3D point 
    R2, C2 : relative camera pose
    Returns:
        x3D : optimized 3D points
    """
    
    P1 = ProjectionMatrix(R1,C1,K) 
    P2 = ProjectionMatrix(R2,C2,K)
    # pts1, pts2, x3D = pts1, pts2, x3D
    
    if pts1.shape[0] != pts2.shape[0] != x3D.shape[0]:
        raise 'Check point dimensions - level nlt'

    x3D_ = []
    for i in range(len(x3D)):
        optimized_params = optimize.least_squares(fun=ReprojectionLoss, x0=x3D[i], method="trf", args=[pts1[i], pts2[i], P1, P2])
        X1 = optimized_params.x
        x3D_.append(X1)
        # x3D_.append(X1[:3])
    return np.array(x3D_)


def ReprojectionLoss(X, pts1, pts2, P1, P2):
    
    # X = homo(X.reshape(1,-1)).reshape(-1,1) # make X a column of homogenous vector
    
    p1_1T, p1_2T, p1_3T = P1 # rows of P1
    p1_1T, p1_2T, p1_3T = p1_1T.reshape(1,-1), p1_2T.reshape(1,-1),p1_3T.reshape(1,-1)

    p2_1T, p2_2T, p2_3T = P2 # rows of P2
    p2_1T, p2_2T, p2_3T = p2_1T.reshape(1,-1), p2_2T.reshape(1,-1), p2_3T.reshape(1,-1)

    ## reprojection error for reference camera points - j = 1
    u1,v1 = pts1[0], pts1[1]
    u1_proj = np.divide(p1_1T.dot(X) , p1_3T.dot(X))
    v1_proj =  np.divide(p1_2T.dot(X) , p1_3T.dot(X))
    E1= np.square(v1 - v1_proj) + np.square(u1 - u1_proj)

    
    ## reprojection error for second camera points - j = 2    
    u2,v2 = pts2[0], pts2[1]
    u2_proj = np.divide(p2_1T.dot(X) , p2_3T.dot(X))
    v2_proj =  np.divide(p2_2T.dot(X) , p2_3T.dot(X))    
    E2= np.square(v2 - v2_proj) + np.square(u2 - u2_proj)
    
    error = E1 + E2
    return error.squeeze()

In [ ]:
new = new / new[:,3][...,None]

In [ ]:
X = NonLinearTriangulation(data.K,pts1, pts2, new,camera_rotations[0], camera_translations[0], R,C )

In [ ]:
(X/X[:,3].reshape(-1,1)).min()

In [ ]:
import scipy.optimize as optimize
import numpy as np

def NonLinearPnP(K, pts, x3D, R0, C0):
    """    
    K : Camera Matrix
    pts1, pts2 : Point Correspondences
    x3D :  initial 3D point 
    R2, C2 : relative camera pose - estimated from PnP
    Returns:
        x3D : optimized 3D points
    """

    Q = Rotation.from_matrix(R0).as_quat()
    X0 = [Q[0] ,Q[1],Q[2],Q[3], C0[0], C0[1], C0[2]] 

    optimized_params = optimize.least_squares(
        fun = PnPLoss,
        x0=X0,
        method="trf",
        args=[x3D, pts, K])
    X1 = optimized_params.x
    Q = X1[:4]
    C = X1[4:]
    R = Rotation.from_quat(Q).as_matrix()
    return R, C

def PnPLoss(X0, x3D, pts, K):
    
    Q, C = X0[:4], X0[4:].reshape(-1,1)
    R = Rotation.from_quat(Q).as_matrix()
    P = computeProjectionMatrix(K,R,C)
    
    Error = []
    for X, pt in zip(x3D, pts):

        p_1T, p_2T, p_3T = P# rows of P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)


        X = to_homogeneous(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
        ## reprojection error for reference camera points 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        E = np.square(v - v_proj) + np.square(u - u_proj)

        Error.append(E)

    sumError = np.mean(np.array(Error).squeeze())
    return sumError

In [ ]:
R,C = DLT_PnP(corr_points, X_new, data.K)

In [ ]:
R

In [ ]:
C

In [ ]:
Rn,Cn = NonLinearPnP(data.K, corr_points, X_new, R,C)

In [ ]:
Rn

In [ ]:
Cn

In [ ]:
R

In [ ]:
C

In [ ]:
X = linearTriangulation(pts1, pts2)

In [ ]:
pts1.shape

In [ ]:
pts2.shape

In [ ]:
pts1

In [ ]:
appended_ids[:3]

In [ ]:
images_to_add[:3]

In [ ]:
appended_ids

In [ ]:
images_to_add

In [ ]:
pts2